In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# Load a sample video from the web (replace with your own video URL)
cap = cv2.VideoCapture('nofi006.mp4')

# Parameters for corner detection
feature_params = dict(maxCorners=500, qualityLevel=0.01, minDistance=10, blockSize=7)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(30, 30), maxLevel=5, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (500, 3))

# Take the first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# Display the images in the notebook
%matplotlib inline

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()

        # Convert coordinates to integers
        pt1 = (int(a), int(b))
        pt2 = (int(c), int(d))

        mask = cv2.line(mask, pt1, pt2, color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

    # Combine the frame with the mask for visualization
    optical_flow_result = cv2.add(frame, mask)

    # Display the optical flow result using cv2_imshow
    cv2_imshow(optical_flow_result)

    # Apply adaptive thresholding for segmentation
    thresholded = cv2.adaptiveThreshold(frame_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Display the segmented frame using cv2_imshow
    cv2_imshow(thresholded)

    k = cv2.waitKey(25)
    if k == 27:
        break

    # Update the previous frame and points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

# Release the video capture object
cap.release()

In [ ]:
from google.colab.patches import cv2_imshow
import cv2 as cv
import numpy as np

# The video feed is read in as
# a VideoCapture object
cap = cv.VideoCapture("nofi006.mp4")

# ret = a boolean return value from
# getting the frame, first_frame = the
# first frame in the entire video sequence
ret, first_frame = cap.read()

# Converts frame to grayscale because we
# only need the luminance channel for
# detecting edges - less computationally
# expensive
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)

# Creates an image filled with zero
# intensities with the same dimensions
# as the frame
mask = np.zeros_like(first_frame)

# Sets image saturation to maximum
mask[..., 1] = 255

while cap.isOpened():
    # ret = a boolean return value from getting
    # the frame, frame = the current frame being
    # projected in the video
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        break

    # Converts each frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculates dense optical flow by Farneback method with adjusted parameters
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray,
                                       None,
                                       pyr_scale=0.5,   # Adjust the image scale
                                       levels=3,
                                       winsize=15,
                                       iterations=3,
                                       poly_n=5,
                                       poly_sigma=1.1,   # Adjust the standard deviation of the Gaussian
                                       flags=0)

    # Computes the magnitude and angle of the 2D vectors
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])

    # Sets image hue according to the optical flow
    # direction
    mask[..., 0] = angle * 180 / np.pi / 2

    # Sets image value according to the optical flow
    # magnitude (normalized)
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)

    # Converts HSV to RGB (BGR) color representation
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

    # Convert the frame to grayscale for adaptive thresholding
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Apply Adaptive Thresholding for segmentation
    thresholded = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)

    # Combine the original frame with the segmented result
    segmented = cv.bitwise_and(frame, frame, mask=thresholded)

    # Concatenate the frames horizontally for display
    display_frame = np.concatenate((frame, rgb, segmented), axis=1)

    # Display the concatenated frame
    cv2_imshow(display_frame)

    # Breaks out of the loop when the user presses the 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    # Updates previous frame
    prev_gray = gray

# Releases resources and closes all windows
cap.release()
cv.destroyAllWindows()
